<a href="https://colab.research.google.com/github/Santapaji/ARORA/blob/main/TASK_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from google.colab import drive
from io import StringIO
import sys
# Loading the data from the CSV files
data = pd.read_csv("/content/sample_data/data.csv")
labels = pd.read_csv("/content/sample_data/label.csv")

# Printing the shape and columns of the data DataFrame to understand its structure
print(data.shape)
print(data.columns)

# Droping the 'unix Timestamp' column as it is not needed for our analysis/model
data = data.drop(columns=['unix Timestamp'])

# Extracting the first 137500 rows of the data.
relevant_data = data.iloc[:137500]

# Reshaping the relevant data to have 55 samples, each of shape (2500, 2) as given in task 1 description
X_data = relevant_data.values.reshape(55, 2500, 2)


(137500, 3)
Index(['unix Timestamp', 'ACC X', 'Light'], dtype='object')


In [8]:
# Splitting data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_data, labels, test_size=0.2)

# Importing necessary layers and utilities
from tensorflow.keras.layers import Dropout, BatchNormalization
from keras.utils import to_categorical

# Encoding labels into one-hot format
y_train_encoded = to_categorical(y_train, num_classes=55)
y_val_encoded = to_categorical(y_val, num_classes=55)

# Initializing the model
model = Sequential()

# Adding the first convolutional layer
model.add(Conv1D(64, kernel_size=3, activation='relu', padding='same', input_shape=(2500, 2)))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))

# Adding the second convolutional layer
model.add(Conv1D(128, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))

# Adding the third convolutional layer
model.add(Conv1D(256, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))

# Adding a fully connected layer
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(55, activation='softmax'))

# Compiling the model with the specified loss function and optimizer
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Re-compiling and training the model (Note: Compiling twice seems redundant and could be improved)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Redirecting stdout to capture model outputs
old_stdout = sys.stdout
sys.stdout = mystdout = StringIO()

# Displaying the model summary
model.summary()

# Training the model and capturing the epoch details
#history = model.fit(X_train, y_train_encoded, validation_data=(X_val, y_val_encoded), epochs=10)
model.fit(X_train, y_train_encoded, validation_data=(X_val, y_val_encoded), epochs=10)
# Restoring the standard output to the original state
sys.stdout = old_stdout

# Extracting the model summary and training outputs
outputs_str = mystdout.getvalue()

# Saving the captured outputs to a text file
with open("output.txt", "w") as f:
    f.write(outputs_str)
